In [ ]:
# Importing Libraries
from pandas import read_csv
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from statsmodels.tsa.seasonal import seasonal_decompose
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
import numpy as np
import keras_tuner as kt
from tensorflow.keras import regularizers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('mergedsensordata.csv')

In [4]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Iterate over all columns except the last one (label column)
for column in df.columns[:-1]:
    # Normalize the column using Min-Max Scaler
    df[column] = scaler.fit_transform(df[[column]])

# Print the normalized data
print(df)


       Unnamed: 0   x_accel   y_accel   z_accel    x_gyro    y_gyro    z_gyro  \
0        0.000000  0.089261  0.367638  0.147360  0.438832  0.620385  0.250587   
1        0.000282  0.533612  0.216018  0.714106  0.669298  0.320921  0.786218   
2        0.000565  0.868619  0.719788  0.318834  0.623915  0.241717  0.303054   
3        0.000847  0.753696  0.403505  0.668085  0.303867  0.683496  0.522318   
4        0.001130  0.551464  0.823925  0.394641  0.108919  0.827075  0.535630   
...           ...       ...       ...       ...       ...       ...       ...   
14160    0.998870  0.360391  0.559405  0.600630  0.502368  0.576838  0.604542   
14161    0.999153  0.282566  0.378847  0.411032  0.535122  0.362575  0.083790   
14162    0.999435  0.574338  0.279804  0.674074  0.488161  0.497002  0.732968   
14163    0.999718  0.576569  0.759323  0.347991  0.695738  0.394131  0.412686   
14164    1.000000  0.437099  0.426330  0.494720  0.255722  0.628905  0.716523   

       temperature  label  

In [5]:
df

,Unnamed: 0,x_accel,y_accel,z_accel,x_gyro,y_gyro,z_gyro,temperature,label
0,0.000000,0.089261,0.367638,0.147360,0.438832,0.620385,0.250587,0.600000,0
1,0.000282,0.533612,0.216018,0.714106,0.669298,0.320921,0.786218,0.568421,0
2,0.000565,0.868619,0.719788,0.318834,0.623915,0.241717,0.303054,0.600000,0
3,0.000847,0.753696,0.403505,0.668085,0.303867,0.683496,0.522318,0.578947,0
4,0.001130,0.551464,0.823925,0.394641,0.108919,0.827075,0.535630,0.578947,0
...,...,...,...,...,...,...,...,...,...
14160,0.998870,0.360391,0.559405,0.600630,0.502368,0.576838,0.604542,0.915789,3
14161,0.999153,0.282566,0.378847,0.411032,0.535122,0.362575,0.083790,0.915789,3
14162,0.999435,0.574338,0.279804,0.674074,0.488161,0.497002,0.732968,0.894737,3
14163,0.999718,0.576569,0.759323,0.347991,0.695738,0.394131,0.412686,0.905263,3


In [6]:
df

,Unnamed: 0,x_accel,y_accel,z_accel,x_gyro,y_gyro,z_gyro,temperature,label
0,0.000000,0.089261,0.367638,0.147360,0.438832,0.620385,0.250587,0.600000,0
1,0.000282,0.533612,0.216018,0.714106,0.669298,0.320921,0.786218,0.568421,0
2,0.000565,0.868619,0.719788,0.318834,0.623915,0.241717,0.303054,0.600000,0
3,0.000847,0.753696,0.403505,0.668085,0.303867,0.683496,0.522318,0.578947,0
4,0.001130,0.551464,0.823925,0.394641,0.108919,0.827075,0.535630,0.578947,0
...,...,...,...,...,...,...,...,...,...
14160,0.998870,0.360391,0.559405,0.600630,0.502368,0.576838,0.604542,0.915789,3
14161,0.999153,0.282566,0.378847,0.411032,0.535122,0.362575,0.083790,0.915789,3
14162,0.999435,0.574338,0.279804,0.674074,0.488161,0.497002,0.732968,0.894737,3
14163,0.999718,0.576569,0.759323,0.347991,0.695738,0.394131,0.412686,0.905263,3


In [7]:
# Assuming your DataFrame is named 'df'

# Drop the 'Unnamed: 0' and 'label' columns
df_updated = df.drop(columns=['Unnamed: 0'])


In [8]:
df_updated

,x_accel,y_accel,z_accel,x_gyro,y_gyro,z_gyro,temperature,label
0,0.089261,0.367638,0.147360,0.438832,0.620385,0.250587,0.600000,0
1,0.533612,0.216018,0.714106,0.669298,0.320921,0.786218,0.568421,0
2,0.868619,0.719788,0.318834,0.623915,0.241717,0.303054,0.600000,0
3,0.753696,0.403505,0.668085,0.303867,0.683496,0.522318,0.578947,0
4,0.551464,0.823925,0.394641,0.108919,0.827075,0.535630,0.578947,0
...,...,...,...,...,...,...,...,...
14160,0.360391,0.559405,0.600630,0.502368,0.576838,0.604542,0.915789,3
14161,0.282566,0.378847,0.411032,0.535122,0.362575,0.083790,0.915789,3
14162,0.574338,0.279804,0.674074,0.488161,0.497002,0.732968,0.894737,3
14163,0.576569,0.759323,0.347991,0.695738,0.394131,0.412686,0.905263,3


In [9]:
def transform_to_lstm_input(data, columns, len_seq_past=1, future=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df_updated = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(len_seq_past, 0, -1):
		cols.append(df_updated.shift(i))
		names += [('%s(t-%d)' % (columns[j], i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, future):
		cols.append(df_updated.shift(-i))
		if i == 0:
			names += [('%s(t)' % (columns[j])) for j in range(n_vars)]
		else:
			names += [('%s(t+%d)' % (columns[j], i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [10]:
len_sequece_past = 60

supervised_series_dataset = transform_to_lstm_input(df_updated,df_updated.columns, len_sequece_past, 1)

In [11]:
supervised_series_dataset.drop([f'label(t-{i})' for i in range(1,len_sequece_past+1)],axis=1, inplace= True)

In [12]:
supervised_series_dataset 

,x_accel(t-60),y_accel(t-60),z_accel(t-60),x_gyro(t-60),y_gyro(t-60),z_gyro(t-60),temperature(t-60),x_accel(t-59),y_accel(t-59),z_accel(t-59),...,z_gyro(t-1),temperature(t-1),x_accel(t),y_accel(t),z_accel(t),x_gyro(t),y_gyro(t),z_gyro(t),temperature(t),label(t)
60,0.089261,0.367638,0.147360,0.438832,0.620385,0.250587,0.600000,0.533612,0.216018,0.714106,...,0.590446,0.578947,0.834868,0.537803,0.472498,0.217443,0.743137,0.231010,0.557895,0
61,0.533612,0.216018,0.714106,0.669298,0.320921,0.786218,0.568421,0.868619,0.719788,0.318834,...,0.231010,0.557895,0.220363,0.764214,0.537589,0.541831,0.691385,0.640564,0.589474,0
62,0.868619,0.719788,0.318834,0.623915,0.241717,0.303054,0.600000,0.753696,0.403505,0.668085,...,0.640564,0.589474,0.110460,0.175871,0.319307,0.418311,0.524140,0.231793,0.568421,0
63,0.753696,0.403505,0.668085,0.303867,0.683496,0.522318,0.578947,0.551464,0.823925,0.394641,...,0.231793,0.568421,0.652720,0.431017,0.621592,0.830702,0.059325,0.799530,0.578947,0
64,0.551464,0.823925,0.394641,0.108919,0.827075,0.535630,0.578947,0.195537,0.668637,0.601418,...,0.799530,0.578947,0.770711,0.382107,0.303073,0.642068,0.418429,0.248238,0.600000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14160,0.280056,0.513756,0.271710,0.524467,0.372988,0.277212,0.884211,0.376569,0.228449,0.824744,...,0.208301,0.905263,0.360391,0.559405,0.600630,0.502368,0.576838,0.604542,0.915789,3
14161,0.376569,0.228449,0.824744,0.685083,0.475860,0.796398,0.884211,0.657183,0.574078,0.161072,...,0.604542,0.915789,0.282566,0.378847,0.411032,0.535122,0.362575,0.083790,0.915789,3
14162,0.657183,0.574078,0.161072,0.492502,0.280846,0.407987,0.894737,0.470572,0.581211,0.779196,...,0.083790,0.915789,0.574338,0.279804,0.674074,0.488161,0.497002,0.732968,0.894737,3
14163,0.470572,0.581211,0.779196,0.582478,0.600189,0.606891,0.905263,0.334728,0.650092,0.378881,...,0.732968,0.894737,0.576569,0.759323,0.347991,0.695738,0.394131,0.412686,0.905263,3


In [13]:
supervised_series_dataset.to_csv('timestep.csv')

In [14]:
np_values = supervised_series_dataset.values
train = np_values[:1500, :]
validation = np_values[1500:1800, :]
test = np_values[1800:, :]
np.random.shuffle(train)
np.random.shuffle(validation)
np.random.shuffle(test)

In [15]:
# train_X is all the rows and column except the last column and train_y is the last column
train_X, train_y = train[:, :-1], tf.keras.utils.to_categorical(train[:, -1],num_classes=4)
validation_X, validation_y = validation[:, :-1], to_categorical(validation[:, -1], num_classes=4)
test_X, test_y = test[:, :-1], to_categorical(test[:, -1])
#np.vectorize(train_y)
#np.vectorize(validation_y)
#np.vectorize(test_y)
print(train_y[0])
print(train_y.shape)
print(train_X.shape)


[1. 0. 0. 0.]
(1500, 4)
(1500, 427)


In [16]:
train_X  = np.reshape(train_X, (train_X .shape[0], 1, train_X .shape[1]))


In [17]:
validation_X = np.reshape(validation_X , (validation_X  .shape[0], 1, validation_X .shape[1]))

In [18]:
test_X = np.reshape(test_X , (test_X  .shape[0], 1, test_X .shape[1]))

In [19]:
print("train_x shape",train_X.shape, "train_y_shape", train_y.shape)

train_x shape (1500, 1, 427) train_y_shape (1500, 4)


In [ ]:

role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::969534953710:role/service-role/AmazonSageMaker-ExecutionRole-20230905T141689


In [21]:
# Create directory to store the training and validation data in the notebook
import os
os.makedirs("./data", exist_ok = True)

# Save several np array into a single file in .npz format
# more infor at https://numpy.org/devdocs/reference/generated/numpy.savez.html

np.savez('./data/training', x = train_X, y = train_y)
np.savez('./data/validation', x =validation_X , y = validation_y)
np.savez('./data/test', x = test_X, y = test_y)

In [22]:
# Upload the training and validation data to S3 bucket

prefix = 'data'
training_input_path = sagemaker_session.upload_data('data/training.npz', key_prefix = prefix + '/training')
validation_input_path = sagemaker_session.upload_data('data/validation.npz', key_prefix = prefix + '/validation')
test_input_path = sagemaker_session.upload_data('data/test.npz', key_prefix = prefix + '/test')

# this paths are used later to fit the model using tf_estimator.fit
print(training_input_path)
print(validation_input_path)
print(test_input_path)

s3://sagemaker-eu-central-1-969534953710/data/training/training.npz
s3://sagemaker-eu-central-1-969534953710/data/validation/validation.npz
s3://sagemaker-eu-central-1-969534953710/data/test/test.npz


In [ ]:
# with use_spot_instance and max_run and max_wait we can reduce significantly the billing.
# this configuration remove the immediate request for instance type with maximum waiting time.

use_spot_instances = True
max_run = 3600
max_wait = 7200 if use_spot_instances else None

from sagemaker.tensorflow import TensorFlow

# To Train a TensorFlow model, we will use TensorFlow estimator from the Sagemaker SDK

# entry_point: a script that will run in a container. This script will include model description and training. 
# role: a role that's obtained The role assigned to the running notebook. 
# train_instance_count: number of container instances used to train the model.
# train_instance_type: instance type!
# framwork_version: version of Tensorflow
# py_version: Python version.
# script_mode: allows for running script in the container. 
# hyperparameters: indicate the hyperparameters for the training job such as epochs and learning rate


tf_estimator = TensorFlow(entry_point='train-cnn.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.large',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          use_spot_instances=use_spot_instances,
                          max_run=max_run,
                          max_wait=max_wait,
                          hyperparameters={
                              'epochs': 1 ,
                              'batch-size': 32,
                              'learning-rate': 0.0001,
                          }
                         )

In [24]:
tf_estimator.fit({'training':training_input_path , 'validation': validation_input_path})

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2023-09-11-11-59-47-262


2023-09-11 11:59:47 Starting - Starting the training job...
2023-09-11 12:00:04 Starting - Preparing the instances for training......
2023-09-11 12:00:59 Downloading - Downloading input data...
2023-09-11 12:01:35 Training - Training image download completed. Training in progress..2023-09-11 12:01:40,170 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2023-09-11 12:01:40,177 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-09-11 12:01:40,374 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-09-11 12:01:40,391 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-09-11 12:01:40,403 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training",
        "validation": "/opt/ml/input/data/validation"
    },
    "current_host": "algo-1"

In [25]:
# once the model has successfully trained, we can deploy it as a end point

import time
tf_endpoint_name = 'lstm-classifier-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_predictor = tf_estimator.deploy(initial_instance_count = 1,
                         instance_type = 'ml.t2.medium',
                         endpoint_name = tf_endpoint_name)     

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: sagemaker-tensorflow-scriptmode-2023-09-11-12-04-00-486
INFO:sagemaker:Creating endpoint-config with name lstm-classifier-2023-09-11-12-04-00
INFO:sagemaker:Creating endpoint with name lstm-classifier-2023-09-11-12-04-00


----!

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Making predictions 

prediction = tf_predictor.predict(test_X)['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)

print('Predicted labels are: {}'.format(predicted_label))
print('Labels are {}'.format(test_y))